In [2]:
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings("ignore")

<h1 style="text-align:center">🏰 Data Modelling</h1> 

<h3 style="text-align:center">Predict BTC prices</h3> 

# 🎖 1. Get newest data points

## ➡️ Define symbol and time interval

In [3]:
symbol = "BTCUSDT"
PERIOD = "15m"

In [4]:
!jupyter nbconvert --to python ../obtain/get_newest_price.ipynb
!python ../obtain/get_newest_price.py

[NbConvertApp] Converting notebook ../obtain/get_newest_price.ipynb to python
[NbConvertApp] Writing 1547 bytes to ../obtain/get_newest_price.py


In [5]:
pd_df = pd.read_csv(f"../../datastore/price/{symbol}_{PERIOD}.csv")
pd_df = pd_df.iloc[-900:]
pd_df

,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
171628,1672434000000,16569.36,16677.35,16568.31,16605.92,5074.86019,1672434899999,8.431324e+07,84269,2554.14607,4.243491e+07
171629,1672434900000,16606.55,16612.64,16586.68,16606.47,1532.46473,1672435799999,2.543646e+07,40517,795.74620,1.320843e+07
171630,1672435800000,16605.85,16615.00,16593.50,16611.23,1196.94665,1672436699999,1.987505e+07,33091,617.61503,1.025562e+07
171631,1672436700000,16611.23,16614.55,16580.20,16585.06,1068.92348,1672437599999,1.773942e+07,34570,476.53479,7.908332e+06
171632,1672437600000,16585.06,16597.00,16584.35,16588.54,1099.59680,1672438499999,1.824181e+07,31056,534.96619,8.874969e+06
...,...,...,...,...,...,...,...,...,...,...,...
172523,1673239500000,17234.00,17248.99,17219.40,17226.49,2324.30990,1673240399999,4.005243e+07,56545,1163.04395,2.004247e+07
172524,1673240400000,17227.11,17236.57,17219.04,17228.37,1669.15254,1673241299999,2.875712e+07,44031,813.24108,1.401120e+07
172525,1673241300000,17228.08,17262.56,17228.08,17235.13,2108.20942,1673242199999,3.634930e+07,53712,1101.33195,1.898917e+07
172526,1673242200000,17235.12,17239.03,17212.30,17213.94,2027.34310,1673243099999,3.492500e+07,45413,930.86570,1.603675e+07


## ➡️ Select features

### 📌 Using Open time as index to visualize later

In [6]:
pd_df["Open price"] = pd_df["Open price"].apply(lambda price: float(price))
pd_df = pd_df.set_index("Kline open time").sort_index() 
pd_df.index = pd.to_datetime(pd_df.index, unit='ms') + pd.Timedelta('07:00:00')
pd_df

Open price  High price  Low price  Close price  \
Kline open time                                                       
2022-12-31 04:00:00    16569.36    16677.35   16568.31     16605.92   
2022-12-31 04:15:00    16606.55    16612.64   16586.68     16606.47   
2022-12-31 04:30:00    16605.85    16615.00   16593.50     16611.23   
2022-12-31 04:45:00    16611.23    16614.55   16580.20     16585.06   
2022-12-31 05:00:00    16585.06    16597.00   16584.35     16588.54   
...                         ...         ...        ...          ...   
2023-01-09 11:45:00    17234.00    17248.99   17219.40     17226.49   
2023-01-09 12:00:00    17227.11    17236.57   17219.04     17228.37   
2023-01-09 12:15:00    17228.08    17262.56   17228.08     17235.13   
2023-01-09 12:30:00    17235.12    17239.03   17212.30     17213.94   
2023-01-09 12:45:00    17217.83    17219.64   17200.86     17206.33   

                         Volume  Kline Close time  Quote asset volume  \
Kline open time                                                         
2022-12-31 04:00:00  5074.86019     1672434899999        8.431324e+07   
2022-12-31 04:15:00  1532.46473     1672435799999        2.543646e+07   
2022-12-31 04:30:00  1196.94665     1672436699999        1.987505e+07   
2022-12-31 04:45:00  1068.92348     1672437599999        1.773942e+07   
2022-12-31 05:00:00  1099.59680     1672438499999        1.824181e+07   
...                         ...               ...                 ...   
2023-01-09 11:45:00  2324.30990     1673240399999        4.005243e+07   
2023-01-09 12:00:00  1669.15254     1673241299999        2.875712e+07   
2023-01-09 12:15:00  2108.20942     1673242199999        3.634930e+07   
2023-01-09 12:30:00  2027.34310     1673243099999        3.492500e+07   
2023-01-09 12:45:00   780.20558     1673243999999        1.342800e+07   

                     Number of trades  Taker buy base asset volume  \
Kline open time                                                      
2022-12-31 04:00:00             84269                   2554.14607   
2022-12-31 04:15:00             40517                    795.74620   
2022-12-31 04:30:00             33091                    617.61503   
2022-12-31 04:45:00             34570                    476.53479   
2022-12-31 05:00:00             31056                    534.96619   
...                               ...                          ...   
2023-01-09 11:45:00             56545                   1163.04395   
2023-01-09 12:00:00             44031                    813.24108   
2023-01-09 12:15:00             53712                   1101.33195   
2023-01-09 12:30:00             45413                    930.86570   
2023-01-09 12:45:00             13217                    382.02312   

                     Taker buy base asset volume  \
Kline open time                                    
2022-12-31 04:00:00                  4.243491e+07  
2022-12-31 04:15:00                  1.320843e+07  
2022-12-31 04:30:00                  1.025562e+07  
2022-12-31 04:45:00                  7.908332e+06  
2022-12-31 05:00:00                  8.874969e+06  
...                                           ...  
2023-01-09 11:45:00                  2.004247e+07  
2023-01-09 12:00:00                  1.401120e+07  
2023-01-09 12:15:00                  1.898917e+07  
2023-01-09 12:30:00                  1.603675e+07  
2023-01-09 12:45:00                  6.574877e+06  

[900 rows x 10 columns]

### 📌 Visualize prices as candle stick

In [7]:
fig = go.Figure(data=[go.Candlestick(x=pd_df.index,
                open=pd_df['Open price'],
                high=pd_df['High price'],
                low=pd_df['Low price'],
                close=pd_df['Close price'])])
# set new height and width
fig.update_layout(
    height=800,
    width=1000,
    title_text="BTC/USDT price",
    yaxis_title="Price (BTC/USDT)",
    xaxis_title="Date",
    xaxis_rangeslider_visible=True
)

fig.show()

### 📌 Using open price as feature and target as well

In [8]:
dataset = pd_df.filter(["Open price"]).values
dataset[:5]

array([[16569.36],
       [16606.55],
       [16605.85],
       [16611.23],
       [16585.06]])

# 🎖 2. Prepare train-test set

✅ Train-Test ratio: `80%` train, `20%` test <br>
✅ Train-Valid ratio: `70%` train, `30%` valid

In [9]:
TRAIN_TEST_LENGTH = int(len(dataset) * 0.8)
TRAIN_VALID_LENGTH = int(TRAIN_TEST_LENGTH * 0.7)

## ➡️ Scale data

In [10]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
scaled_data[:5]

array([[0.09090661],
       [0.14160112],
       [0.14064694],
       [0.14798053],
       [0.11230763]])

## ➡️ Prepare time series data

In [11]:
def get_split_data(start, end = None):
    data = scaled_data[start:end]
    X = []
    Y = []
    for i in range(60,len(data)):
        X.append(data[i-60:i,:])
        Y.append(data[i,:])

    X, Y = np.array(X), np.array(Y)
    return X,Y

## ➡️ Split train-test set

In [12]:
X_origin_train, Y_origin_train = get_split_data(0, TRAIN_TEST_LENGTH)
X_test, Y_test = get_split_data(TRAIN_TEST_LENGTH-60, )

print(X_origin_train.shape, Y_origin_train.shape)
print(X_test.shape, Y_test.shape)

(660, 60, 1) (660, 1)
(180, 60, 1) (180, 1)


## ➡️ Split train-valid set

In [13]:
X_train, Y_train = get_split_data(0, TRAIN_VALID_LENGTH)
X_valid, Y_valid = get_split_data(TRAIN_VALID_LENGTH-60, TRAIN_TEST_LENGTH)

print(X_train.shape, Y_train.shape)
print(X_valid.shape, Y_valid.shape)

(443, 60, 1) (443, 1)
(217, 60, 1) (217, 1)


# 🎖 3. Naive training

## ➡️ Visualize predictions

In [14]:
def visualize_prediction(preds):
    data = pd_df[["Open price"]]
    train = data[:TRAIN_VALID_LENGTH]
    valid = data[TRAIN_VALID_LENGTH:TRAIN_TEST_LENGTH].reset_index()
    valid["Predict"] = preds[:, 0]
    valid = valid.set_index("Kline open time")
    valid["Predict"]= valid["Predict"].apply(lambda price: float(price))

    concat_df = pd.concat([train, valid], axis=0)
    fig = px.line(concat_df[["Open price","Predict"]], title="BTC/USDT price" , width=1000, height=800)
    fig.show()

## ➡️ Linear Regression

### 📌 Build model and train

## ➡️ Linear Regression

### 📌 Build model and train

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
model = LinearRegression()
model.fit(X_train.reshape(X_train.shape[0], X_train.shape[1]), Y_train)

LinearRegression()

### 📌 Predict and transform to the original scale

In [17]:
predictions = model.predict(X_valid.reshape(X_valid.shape[0], X_valid.shape[1]))
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
predictions.shape

(217, 1)

### 📌 Visualize the prediction

In [18]:
visualize_prediction(predictions)

## ➡️ Gated Recurrent Unit (GRU)

### 📌 Build model and train

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM

2023-01-09 12:47:59.324773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(100))
model.add(Dense(X_train.shape[2]))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, Y_train, batch_size=1, epochs=1)

2023-01-09 12:48:02.465509: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


443/443 [==============================] - 8s 14ms/step - loss: 0.0029


### 📌 Predict and transform to the original scale

In [21]:
predictions = model.predict(X_valid)
predictions = scaler.inverse_transform(predictions)
predictions.shape

7/7 [==============================] - 1s 20ms/step


(217, 1)

### 📌 Visualize the prediction

In [22]:
visualize_prediction(predictions)

## ➡️ Seasonal Auto Regressive Integrated Moving Average (SARIMAX)

In [23]:
from statsmodels.tsa.api import SARIMAX

### 📌 Build model and train

In [ ]:
predictions = list()
for t in range(X_valid.shape[0]):
    model = SARIMAX(X_valid[t])
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)

### 📌 Predict and transform to the original scale

In [25]:
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
predictions.shape

(217, 1)

### 📌 Visualize the prediction

In [26]:
visualize_prediction(predictions)        

# 🎖 4. Model selection and evaluation

In [27]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, r2_score
from estimator import LSTMEstimator, SarimaxEstimator, LinearRegressionEstimator

## ➡️ Naive cross validation to select model

### 📌 Cross validation method

In [28]:
tscv = TimeSeriesSplit(n_splits=3)

- The method that can be used for cross-validating the time-series model is cross-validation on a rolling basis. Start with a small subset of data for training purpose, forecast for the later data points and then checking the accuracy for the forecasted data points. The same forecasted data points are then included as part of the next training dataset and subsequent data points are forecasted. <br>
![](https://miro.medium.com/max/640/1*XcqvKVTQ6U_zszSD52lSqA.webp)

### 📌 Metric method

In [29]:
metric = make_scorer(r2_score)

- $R^2$ score is used to evaluate the performance of a linear regression model. It is the amount of the variation in the output dependent attribute which is predictable from the input independent variable(s) <br>
![](https://vitalflux.com/wp-content/uploads/2019/07/R-squared-formula-function-of-SSE-and-SST.jpg)

### 📌 Evaluate models

In [30]:
estimators = [
    LinearRegressionEstimator(),
    LSTMEstimator(), 
    SarimaxEstimator(), 
]

In [ ]:
scores_dict = {}

for estimator in estimators:
    scores = cross_val_score(estimator, X_origin_train, Y_origin_train, scoring=metric, cv=tscv, n_jobs=-1)
    print(estimator.__class__.__name__, scores.mean())
    scores_dict[estimator.__class__.__name__] = scores
    
scores_df = pd.DataFrame(scores_dict)
scores_df.index = scores_df.index.map(lambda x: f'fold_{x+1}')
scores_df.loc['mean'] = scores_df.mean()

### 📌 Score and reveal best model

In [32]:
scores_df

,LinearRegressionEstimator,LSTMEstimator,SarimaxEstimator
fold_1,0.700039,0.287115,0.686077
fold_2,0.862796,0.687337,0.807752
fold_3,0.942435,0.821581,0.914218
mean,0.835090,0.598678,0.802682


- The best best_estimator is one that have highest mean of scores

In [33]:
best_estimator = scores_df.loc['mean'].idxmin()
best_estimator

'LSTMEstimator'

## ➡️ Select best model with best hyperparameters using grid search

In [34]:
from estimator import Estimator, LinearRegressionEstimator, SarimaxEstimator, LSTMEstimator 
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline

### 📌 Cross validation and metric method

- Using aforementioned methods

In [35]:
tscv = TimeSeriesSplit()
metric = make_scorer(r2_score)

### 📌 Evaluate hyperparameters and models 

In [36]:
parameters = [
    {
        'reg__estimator': [LinearRegressionEstimator()]
    },
    {
        'reg__estimator': [SarimaxEstimator()],
        'reg__estimator__order': [(1, 1, 1), (3, 1, 1)],
        'reg__estimator__seasonal_order': [(1, 1, 1, 12), (3, 1, 1, 12)]
    },
    {
        'reg__estimator': [LSTMEstimator()],
        'reg__estimator__epochs': [1, 5, 10],
        'reg__estimator__batch_size': [4, 16, 32],
        'reg__estimator__neurons': [50, 100, 200]
    },
]

In [37]:
pipeline = Pipeline(
    steps=[("reg", Estimator())]
)

grid_search = GridSearchCV(pipeline, parameters, scoring=metric, cv=tscv, n_jobs=2, verbose=0)
grid_search.fit(X_train, Y_train)

75/75 [==============================] - 2s 24ms/step - loss: 7.6342e-04
Epoch 5/10
93/93 [==============================] - 5s 23ms/step - loss: 0.0038
Epoch 2/10
93/93 [==============================] - 2s 22ms/step - loss: 0.0013
Epoch 3/10
75/75 [==============================] - 2s 27ms/step - loss: 6.1075e-04
Epoch 7/10
93/93 [==============================] - 3s 27ms/step - loss: 9.6509e-04
Epoch 4/10
75/75 [==============================] - 2s 25ms/step - loss: 5.9477e-04
Epoch 8/10
93/93 [==============================] - 2s 25ms/step - loss: 8.7371e-04
Epoch 5/10
75/75 [==============================] - 2s 26ms/step - loss: 6.3030e-04
Epoch 9/10
93/93 [==============================] - 2s 24ms/step - loss: 8.0861e-04
Epoch 6/10
75/75 [==============================] - 2s 23ms/step - loss: 5.5242e-04
Epoch 10/10
93/93 [==============================] - 2s 24ms/step - loss: 6.8007e-04
Epoch 7/10
93/93 [==============================] - 2s 19ms/step - loss: 6.6475e-04
Epoch 8/10

2023-01-09 13:05:13.793752: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


14/14 [==============================] - 2s 54ms/step - loss: 0.0048


2023-01-09 13:05:17.023867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


3/3 [==============================] - 0s 17ms/step
Epoch 1/5
5/5 [==============================] - 3s 24ms/step - loss: 0.0028
Epoch 2/5
5/5 [==============================] - 0s 18ms/step - loss: 9.8104e-04
Epoch 3/5
5/5 [==============================] - 0s 20ms/step - loss: 0.0012
Epoch 4/5
5/5 [==============================] - 0s 22ms/step - loss: 8.2475e-04
Epoch 5/5
3/3 [==============================] - 1s 19ms/step
Epoch 1/5
3/3 [==============================] - 0s 6ms/step
Epoch 1/5
14/14 [==============================] - 2s 14ms/step - loss: 0.0064
Epoch 2/5
14/14 [==============================] - 0s 13ms/step - loss: 0.0022
Epoch 3/5
10/10 [==============================] - 2s 20ms/step - loss: 0.0032
Epoch 2/5
14/14 [==============================] - 0s 14ms/step - loss: 0.0013
Epoch 4/5
10/10 [==============================] - 0s 22ms/step - loss: 0.0012
Epoch 3/5
14/14 [==============================] - 0s 17ms/step - loss: 0.0011
Epoch 5/5
10/10 [==================

3/3 [==============================] - 0s 14ms/step
Epoch 1/5
14/14 [==============================] - 2s 22ms/step - loss: 0.0073
Epoch 2/5
14/14 [==============================] - 0s 22ms/step - loss: 0.0013
Epoch 3/5
14/14 [==============================] - 0s 20ms/step - loss: 9.0020e-04
Epoch 4/5
14/14 [==============================] - 0s 20ms/step - loss: 7.6720e-04
Epoch 5/5
19/19 [==============================] - 3s 22ms/step - loss: 0.0078
Epoch 2/5
19/19 [==============================] - 0s 24ms/step - loss: 0.0013
Epoch 3/5
19/19 [==============================] - 0s 22ms/step - loss: 0.0010
Epoch 4/5
19/19 [==============================] - 0s 24ms/step - loss: 9.0965e-04
Epoch 5/5
19/19 [==============================] - 0s 23ms/step - loss: 7.7040e-04


3/3 [==============================] - 0s 10ms/step
Epoch 1/5
24/24 [==============================] - 3s 23ms/step - loss: 0.0113
Epoch 2/5
24/24 [==============================] - 1s 22ms/step - loss: 0.0016
Epoch 3/5
24/24 [==============================] - 1s 24ms/step - loss: 0.0014
Epoch 4/5
24/24 [==============================] - 1s 23ms/step - loss: 0.0013
Epoch 5/5
5/5 [==============================] - 3s 47ms/step - loss: 0.0026
Epoch 2/5
5/5 [==============================] - 0s 59ms/step - loss: 9.4933e-04
Epoch 3/5
5/5 [==============================] - 0s 47ms/step - loss: 8.7337e-04
Epoch 4/5
5/5 [==============================] - 0s 48ms/step - loss: 8.2595e-04
Epoch 5/5
3/3 [==============================] - 0s 18ms/step
Epoch 1/5
Epoch 1/5
10/10 [==============================] - 3s 56ms/step - loss: 0.0015
Epoch 2/5
14/14 [==============================] - 3s 54ms/step - loss: 0.0049
Epoch 2/5
10/10 [==============================] - 1s 60ms/step - loss: 6.3486e-04

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('reg', Estimator())]), n_jobs=2,
             param_grid=[{'reg__estimator': [LinearRegressionEstimator()]},
                         {'reg__estimator': [SarimaxEstimator(order=(1, 1, 1))],
                          'reg__estimator__order': [(1, 1, 1), (3, 1, 1)],
                          'reg__estimator__seasonal_order': [(1, 1, 1, 12),
                                                             (3, 1, 1, 12)]},
                         {'reg__estimator': [LSTMEstimator()],
                          'reg__estimator__batch_size': [4, 16, 32],
                          'reg__estimator__epochs': [1, 5, 10],
                          'reg__estimator__neurons': [50, 100, 200]}],
             scoring=make_scorer(r2_score))

- Best model with best hyperparameters

In [38]:
grid_search.best_params_

{'reg__estimator': SarimaxEstimator(order=(1, 1, 1)),
 'reg__estimator__order': (1, 1, 1),
 'reg__estimator__seasonal_order': (1, 1, 1, 12)}

# 🎖 5. Predict future prices

- We utilize the best model to predict prices

In [ ]:
from statsmodels.tsa.api import SARIMAX

data = pd_df.filter(["Open price"])

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data).reshape(-1)

predictions = list()
model = SARIMAX(scaled_data, order=(1,1,1), seasonal_order=(1,1,1,12))
model_fit = model.fit()
output = model_fit.forecast()
pred_price = output[0]
predictions.append(pred_price)

In [40]:
pred_price = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
pred_price

array([[17218.5754658]])